<a href="https://colab.research.google.com/github/fbelleau/kibio-bi/blob/main/Demo_case_2_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Eland librairy compatible with actual Kibio.science version

In [1]:
!pip install eland==7.14.0b1

import eland as ed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 KB 25.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


## Install most recent version of Pandas (default on Collab is 1.3.5)

In [2]:


import pandas as pd

pd.__version__

'1.3.5'

## Import Elasticsearch's needed librairies

In [3]:
from elasticsearch import Elasticsearch, helpers
import matplotlib.pyplot as plt

## Create connection to Kibio.science's Elastiseach REST endpoint

In [4]:
es = Elasticsearch("http://es.kibio.science:80", verify_certs=False, ssl_show_warn=False)
es

<Elasticsearch([{'host': 'es.kibio.science', 'port': 80}])>

## Dowload needed dataframes from Kibio.science using Elasticsearch's Eland librairy

In [20]:
# download nodes

disease = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_disease_201811'))
print("disease", disease.shape)
gene = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_gene_201811'))
print("gene", gene.shape)
compound = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_compound_201811'))
print("compound", compound.shape)
biological_process = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_biological_process_201811'))
print("biological_process", biological_process.shape)

disease (137, 17)
gene (20945, 17)
compound (1552, 20)
biological_process (11381, 11)


In [22]:
# edges_gene_regulates

gene_regulates = gene[['@id','regulates.@id']]
gene_regulates_edges = gene_regulates.explode('regulates.@id')
gene_regulates_edges.shape

(281983, 2)

In [24]:
# edges_disease_association

disease_association = disease[['@id','associates.@id']]
disease_association_edges = disease_association.explode('associates.@id')
disease_association_edges.shape

(12626, 2)

In [36]:
# download edges

edges_disease_associates = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_edges_disease_associates'))
print("edges_disease_associates", edges_disease_associates.shape)
edges_gene_interacts = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_edges_gene_interacts'))
print("edges_gene_interacts", edges_gene_interacts.shape)
edges_gene_participates = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_edges_gene_participates'))
print("edges_gene_participates", edges_gene_participates.shape)
edges_gene_regulates = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_edges_gene_regulates'))
print("edges_gene_regulates", edges_gene_regulates.shape)
edges_compound_binds = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_edges_compound_binds'))
print("edges_compound_binds", edges_compound_binds.shape)

edges_disease_associates (12626, 6)
edges_gene_interacts (158583, 6)
edges_gene_participates (819199, 6)
edges_gene_regulates (281983, 6)
edges_compound_binds (11734, 6)


## Question 1

In [33]:
gene_regulates_gene = gene_regulates_edges[['@id', 'regulates.@id']].groupby(['@id'],as_index=False).agg({'regulates.@id': lambda x: x.tolist()})
gene_regulates_gene.shape

(20945, 2)

In [34]:
answer_question_1 = gene_regulates_gene[gene_regulates_gene['regulates.@id'].apply(len) > 15]

answer_question_1.shape

(3599, 2)

In [47]:
answer_question_1 = gene[gene['regulates.@id'] == gene['regulates.@id']]
answer_question_1.shape

answer_question_1_2 = answer_question_1[['@id', 'regulates.@id']]
answer_question_1_2.shape

(4634, 2)

In [48]:
answer_question_1_3 = answer_question_1_2[answer_question_1_2['regulates.@id'].apply(len) > 15]

answer_question_1_3.shape

(3599, 2)

In [37]:
# version Pascal

gene_regulates_gene = edges_gene_regulates[['@id', '@id_to']].groupby(['@id'],as_index=False).agg({'@id_to': lambda x: x.tolist()})
gene_regulates_gene.columns = ['gene_1', 'gene_2_list']
answer_question_1 = gene_regulates_gene[gene_regulates_gene['gene_2_list'].apply(len) > 15]

answer_question_1.shape

(3599, 2)

## Question 2

In [35]:
compound_gene = pd.merge(compound[['@id', 'name']], edges_compound_binds[['@id','@id_to']], on='@id')
compound_gene.columns = ['compound_id', 'compound_name', 'gene_id']

compound_gene_disease = pd.merge(compound_gene, edges_disease_associates[['@id', '@id_to']], left_on='gene_id', right_on='@id_to').drop(['@id_to'], axis=1)
compound_gene_disease.columns = ['compound_id', 'compound_name', 'gene_id', 'disease_id']

compound_gene_disease = pd.merge(compound_gene_disease, disease[['@id','name']], left_on='disease_id', right_on='@id', how='inner').drop(['@id'], axis=1)
compound_gene_disease.columns = ['compound_id', 'compound_name', 'gene_id', 'disease_id', 'disease_name']

compound_gene_disease = compound_gene_disease.query('compound_name.str.contains("Valproic Acid")').groupby(['disease_name'],as_index=False).agg({'gene_id': lambda x: x.tolist()})
compound_gene_disease['num_of_genes'] = compound_gene_disease['gene_id'].str.len()

answer_question_2 = compound_gene_disease.sort_values(by=['num_of_genes'], ascending=False).head(n=3)

answer_question_2


NameError: ignored

## Download edges

## Question 3

In [ ]:
disease_gene1 = pd.merge(disease[['@id','name']], edges_disease_associates[['@id','@id_to']], on='@id')
disease_gene1.columns = ['disease_id', 'disease_name', 'gene_1_id']

disease_gene1_gene_2 = pd.merge(disease_gene1, edges_gene_interacts[['@id', '@id_to']], left_on = 'gene_1_id', right_on = '@id').drop(['@id'], axis=1)
disease_gene1_gene_2.columns = ['disease_id', 'disease_name', 'gene_1_id', 'gene_2_id']

disease_gene1_gene_2_process = pd.merge(disease_gene1_gene_2, edges_gene_participates[['@id', '@id_to']], left_on = 'gene_2_id', right_on = '@id').drop(['@id'], axis=1)
disease_gene1_gene_2_process.columns = ['disease_id', 'disease_name', 'gene_1_id', 'gene_2_id', 'process_id']
disease_gene1_gene_2_process = pd.merge(disease_gene1_gene_2_process, biological_process[['@id', 'name']], left_on = 'process_id', right_on = '@id').drop(['@id'], axis=1)
disease_gene1_gene_2_process.columns = ['disease_id', 'disease_name', 'gene_1_id', 'gene_2_id', 'process_id', 'process_name']

answer_question_3 = disease_gene1_gene_2_process.query('disease_name.str.contains("multiple sclerosis") & process_name.str.contains("retina layer formation")')

answer_question_3